In [ ]:
import os 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dataframe_image as dfi

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import ttest_rel
from scipy.stats import shapiro
from scipy.stats import kruskal

import lifelines as lf
from lifelines import CoxPHFitter

import statsmodels.api as sm
from statsmodels.stats.contingency_tables import mcnemar




In [ ]:
we=pd.read_excel('.xlsx')

In [ ]:
# we['중도탈락 여부'].value_counts()

In [ ]:
# '중도탈락 여부' 컬럼에 값이 있는 행 제거
df = we[we['중도탈락 여부'].isna()]

df = df.dropna(subset=['No'])

df['gender']= df['gender'].map({'남': 1, '여': 0})

df.columns = [col.replace('.', '_') for col in df.columns]

In [ ]:
# 데이터프레임 생성
df = pd.DataFrame(df)

# 컬럼 이름 부분적으로 변경
df= df.rename(columns={
    'SBP': 'Visit1_SBP',
    'SBP_1': 'Visit2_SBP',
    'SBP_2':'Visit3_SBP',
    'SBP_3': 'Visit4_SBP',
    'SBP_4': 'Visit5_SBP',
    'HR': 'Visit1_HR',
    'HR_1': 'Visit2_HR',
    'HR_2':'Visit3_HR',
    'HR_3': 'Visit4_HR',
    'HR_4': 'Visit5_HR',
    'DBP': 'Visit1_DBP',
    'DBP_1': 'Visit2_DBP',
    'DBP_2':'Visit3_DBP',
    'DBP_3': 'Visit4_DBP',
    'DBP_4': 'Visit5_DBP',
    'platelet_1': 'Visit1_PLT',
    'platelet': 'Visit5_PLT',
    'WBC_1' : 'Visit1_WBC',
    'WBC_2': 'Visit5_WBC',
    'Hct_1': 'Visit1_HCT',
    'Hct_2': 'Visit5_HCT',
    'Hb_1': 'Visit1_HB',
    'Hb_2': 'Visit5_HB',
    'heartburn':'Visit1_heartburn',
    'heartburn_1':'Visit2_heartburn',
    'heartburn_2':'Visit3_heartburn',
    'heartburn_3':'Visit4_heartburn',
    'heartburn_4':'Visit5_heartburn',
    'nausea': 'Visit1_nausea',
    'nausea_1': 'Visit2_nausea',
    'nausea_2': 'Visit3_nausea',
    'nausea_3': 'Visit4_nausea',
    'nausea_4': 'Visit5_nausea',
    'vomiting': 'Visit1_vomiting',
    'vomiting_1': 'Visit2_vomiting',
    'vomiting_2': 'Visit3_vomiting',
    'vomiting_3': 'Visit4_vomiting',
    'vomiting_4': 'Visit5_vomiting',
    'diarrhea': 'Visit1_diarrhea',
    'diarrhea_1': 'Visit2_diarrhea',
    'diarrhea_2': 'Visit3_diarrhea',
    'diarrhea_3': 'Visit4_diarrhea',
    'diarrhea_4': 'Visit5_diarrhea',
    'abdominal pain' : 'Visit1_abdominal_pain',
    'abdominal pain_1' : 'Visit2_abdominal_pain',
    'abdominal pain_2' : 'Visit3_abdominal_pain',
    'abdominal pain_3' : 'Visit4_abdominal_pain',
    'abdominal pain_4' : 'Visit5_abdominal_pain',
    'dyspepsia': 'Visit1_dyspepsia',
    'dyspepsia_1': 'Visit2_dyspepsia',
    'dyspepsia_2': 'Visit3_dyspepsia',
    'dyspepsia_3': 'Visit4_dyspepsia',
    'dyspepsia_4': 'Visit5_dyspepsia',
    'other': 'Visit1_other',
    'other_1': 'Visit2_other',
    'other_2': 'Visit3_other',
    'other_3': 'Visit4_other',
    'other_4': 'Visit5_other',
    'hematochezia': 'Visit1_hematochezia',
    'hematochezia_1': 'Visit2_hematochezia',
    'hematochezia_2': 'Visit3_hematochezia',
    'hematochezia_3': 'Visit4_hematochezia',
    'hematochezia_4': 'Visit5_hematochezia',
    'melena': 'Visit1_melena',
    'melena_1': 'Visit2_melena',
    'melena_2': 'Visit3_melena',
    'melena_3': 'Visit4_melena',
    'melena_4': 'Visit5_melena',
    'hematemesis': 'Visit1_hematemesis',
    'hematemesis_1': 'Visit2_hematemesis',
    'hematemesis_2': 'Visit3_hematemesis',
    'hematemesis_3': 'Visit4_hematemesis',
    'hematemesis_4': 'Visit5_hematemesis',
    # 'hospitalization': 'Visit1_hospitalization',
    # 'hospitalization_1': 'Visit2_hospitalization',
    # 'hospitalization_2': 'Visit3_hospitalization',
    # 'hospitalization_3': 'Visit4_hospitalization',
    # 'hospitalization_4': 'Visit5_hospitalization',
})



In [ ]:
# 오타 수정 
df.loc[df['Visit5_HB'] == 23.2, 'Visit5_HB'] = 13.2
df.loc[df['Visit1_melena'] == 2009.0, 'Visit1_melena'] = 0
df.loc[df['Visit1_hematemesis'] == 2009.0, 'Visit1_hematemesis'] = 0

In [ ]:
# 평균과 신뢰구간 계산 함수
def mean_confidence_interval(data, confidence=0.95):
    n = len(data)
    m, se = np.mean(data), stats.sem(data)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

# 통계 검정 수행 함수
def perform_tests(group1, group2, label):
    print(f"\nPaired t-test ({label}):")
    stat, p = stats.ttest_rel(group1, group2)
    print(f'Paired t-test : Statistics={stat:.5f}, p={p:.5f}')
    print(f'p-value interpretation: {p_value_stars(p)}')

    print(f"\nWilcoxon signed-rank test ({label}):")
    stat, p = stats.wilcoxon(group1, group2)
    print(f'Wilcoxon signed-rank test: Statistics={stat:.5f}, p={p:.5f}')
    print(f'p-value interpretation: {p_value_stars(p)}')

def p_value_stars(p_value):
    if p_value < 0.001:
        return 'p < 0.001***'
    elif p_value < 0.01:
        return 'p < 0.01**'
    elif p_value < 0.05:
        return 'p < 0.05*'
    else:
        return f"p = {p_value:.2f}"

# 데이터 전처리 및 그룹 나누기 함수
def process_data(df, column):
    cleaned_df_total = df[[f'Visit1_{column}', f'Visit5_{column}']].dropna()
    mean_threshold = df[f'Visit1_{column}'].mean()

    high_group = df[df[f'Visit1_{column}'] > mean_threshold].copy()
    low_group = df[df[f'Visit1_{column}'] <= mean_threshold].copy()

    high_group.loc[:, f'{column}_diff'] = high_group[f'Visit1_{column}'] - high_group[f'Visit5_{column}']
    low_group.loc[:, f'{column}_diff'] = low_group[f'Visit1_{column}'] - low_group[f'Visit5_{column}']

    return cleaned_df_total, high_group, low_group

# 시각화 함수
def visualize_data(df, high_group, low_group, column):
    # 전체 데이터 변화 시각화
    plt.figure(figsize=(8, 12))

    total_means = [df[f'Visit1_{column}'].mean(), df[f'Visit5_{column}'].mean()]
    total_ci = [mean_confidence_interval(df[f'Visit1_{column}']), mean_confidence_interval(df[f'Visit5_{column}'])]

    high_means = [high_group[f'Visit1_{column}'].mean(), high_group[f'Visit5_{column}'].mean()]
    high_ci = [mean_confidence_interval(high_group[f'Visit1_{column}']), mean_confidence_interval(high_group[f'Visit5_{column}'])]

    low_means = [low_group[f'Visit1_{column}'].mean(), low_group[f'Visit5_{column}'].mean()]
    low_ci = [mean_confidence_interval(low_group[f'Visit1_{column}']), mean_confidence_interval(low_group[f'Visit5_{column}'])]
    
    plt.subplot(2, 1, 1)
    plt.errorbar(['Visit1', 'Visit5'], total_means, yerr=[(ci[2]-ci[1])/2 for ci in total_ci], fmt='-o', label=f'Total {column}', color='green', capsize=5)
    plt.errorbar(['Visit1', 'Visit5'], high_means, yerr=[(ci[2]-ci[1])/2 for ci in high_ci], fmt='-o', label=f'High {column}', color='blue', capsize=5)
    plt.errorbar(['Visit1', 'Visit5'], low_means, yerr=[(ci[2]-ci[1])/2 for ci in low_ci], fmt='-o', label=f'Low {column}', color='red', capsize=5)
    plt.title(f'Total Mean {column} Levels with 95% Confidence Intervals')
    plt.ylabel(f'{column} Level')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # 변화량 시각화
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    sns.histplot(high_group[f'{column}_diff'], bins=10, kde=True, color='blue')
    plt.title(f'High {column} Group - {column}_diff')
    plt.xlabel(f'{column}_diff (Visit1_{column} - Visit5_{column})')
    plt.ylabel('Count')

    plt.subplot(1, 2, 2)
    sns.histplot(low_group[f'{column}_diff'], bins=10, kde=True, color='red')
    plt.title(f'Low {column} Group - {column}_diff')
    plt.xlabel(f'{column}_diff (Visit1_{column} - Visit5_{column})')
    plt.ylabel('Count')

    plt.tight_layout()
    plt.show()

    # 그룹별 변화 비교 시각화
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(['Visit1', 'Visit5'], [high_group[f'Visit1_{column}'].mean(), high_group[f'Visit5_{column}'].mean()], marker='o', label=f'High {column}', color='blue')
    plt.plot(['Visit1', 'Visit5'], [low_group[f'Visit1_{column}'].mean(), low_group[f'Visit5_{column}'].mean()], marker='o', label=f'Low {column}', color='red')
    plt.title(f'Mean {column} Levels by Group')
    plt.ylabel(f'{column} Level')
    plt.legend()

    plt.subplot(1, 2, 2)
    sns.boxplot(data=[high_group[f'{column}_diff'], low_group[f'{column}_diff']], palette=['blue', 'red'])
    plt.xticks([0, 1], [f'High {column}', f'Low {column}'])
    plt.title(f'{column}_diff Box Plot by Group')
    plt.ylabel(f'{column}_diff (Visit1_{column} - Visit5_{column})')

    plt.tight_layout()
    plt.show()

# 분석 및 시각화 수행 함수
def analyze_and_visualize(df, column):
    cleaned_df_total, high_group, low_group = process_data(df, column)
    
    perform_tests(df[f'Visit1_{column}'], df[f'Visit5_{column}'], 'Total')
    perform_tests(high_group[f'Visit1_{column}'], high_group[f'Visit5_{column}'], f'High {column}')
    perform_tests(low_group[f'Visit1_{column}'], low_group[f'Visit5_{column}'], f'Low {column}')

    visualize_data(df, high_group, low_group, column)

    # 기초 통계 분석 출력
    print(f"\nTotal {column} Group Statistics:")
    print(cleaned_df_total.describe())

    print(f"\nHigh {column} Group Statistics:")
    print(high_group[[f'Visit1_{column}', f'Visit5_{column}']].describe())

    print(f"\nLow {column} Group Statistics:")
    print(low_group[[f'Visit1_{column}', f'Visit5_{column}']].describe())

# WBC에 대해 분석 및 시각화 수행
analyze_and_visualize(df, 'PLT')


In [ ]:
# 소화기관 출혈 데이터 추출
bleeding_cols = [
    'Visit1_hematochezia', 'Visit1_melena', 'Visit1_hematemesis',
    'Visit2_hematochezia', 'Visit2_melena', 'Visit2_hematemesis',
    'Visit3_hematochezia', 'Visit3_melena', 'Visit3_hematemesis',
    'Visit4_hematochezia', 'Visit4_melena', 'Visit4_hematemesis',
    'Visit5_hematochezia', 'Visit5_melena', 'Visit5_hematemesis'
]

bleeding_data = df[bleeding_cols]

# 각 방문별 출혈 발생 빈도 계산
bleeding_summary = {
    "Visit1": bleeding_data[['Visit1_hematochezia', 'Visit1_melena', 'Visit1_hematemesis']].sum().sum(),
    "Visit2": bleeding_data[['Visit2_hematochezia', 'Visit2_melena', 'Visit2_hematemesis']].sum().sum(),
    "Visit3": bleeding_data[['Visit3_hematochezia', 'Visit3_melena', 'Visit3_hematemesis']].sum().sum(),
    "Visit4": bleeding_data[['Visit4_hematochezia', 'Visit4_melena', 'Visit4_hematemesis']].sum().sum(),
    "Visit5": bleeding_data[['Visit5_hematochezia', 'Visit5_melena', 'Visit5_hematemesis']].sum().sum()
}

bleeding_summary_df = pd.DataFrame(list(bleeding_summary.items()), columns=['Visit', 'Total_Bleeding'])

# 출혈 발생 빈도 그래프 그리기
plt.figure(figsize=(10, 6))
plt.plot(bleeding_summary_df['Visit'], bleeding_summary_df['Total_Bleeding'], marker='o', linestyle='-', color='red')
plt.title('Total Bleeding Events per Visit')
plt.xlabel('Visit')
plt.ylabel('Total Bleeding Events')
plt.grid(True)
plt.show()


In [ ]:
# 소화기관 증상 데이터 추출
symptoms_cols = [
    'Visit1_heartburn', 'Visit1_nausea', 'Visit1_vomiting', 'Visit1_diarrhea', 'Visit1_abdominal_pain', 'Visit1_dyspepsia',
    'Visit2_heartburn', 'Visit2_nausea', 'Visit2_vomiting', 'Visit2_diarrhea', 'Visit2_abdominal_pain', 'Visit2_dyspepsia',
    'Visit3_heartburn', 'Visit3_nausea', 'Visit3_vomiting', 'Visit3_diarrhea', 'Visit3_abdominal_pain', 'Visit3_dyspepsia',
    'Visit4_heartburn', 'Visit4_nausea', 'Visit4_vomiting', 'Visit4_diarrhea', 'Visit4_abdominal_pain', 'Visit4_dyspepsia',
    'Visit5_heartburn', 'Visit5_nausea', 'Visit5_vomiting', 'Visit5_diarrhea', 'Visit5_abdominal_pain', 'Visit5_dyspepsia'
]

symptoms_data = df[symptoms_cols]

# 증상 발생 총합 계산
visit_totals = []
for i in range(1, 6):
    visit_symptoms = symptoms_data.iloc[:, (i-1)*6:i*6].sum(axis=1)
    visit_totals.append(visit_symptoms.sum())

# 결과를 데이터프레임으로 정리
symptoms_total_df = pd.DataFrame({
    'Visit': [f'Visit{i}' for i in range(1, 6)],
    'Total_Symptoms': visit_totals
})

# 증상 총합 그래프 그리기
plt.figure(figsize=(10, 6))
plt.plot(symptoms_total_df['Visit'], symptoms_total_df['Total_Symptoms'], marker='o', linestyle='-', color='skyblue')
plt.title('Total Symptoms per Visit')
plt.xlabel('Visit')
plt.ylabel('Total Symptoms')
plt.grid(True)
plt.show()


In [ ]:
# 소화기관 증상 데이터 추출
symptoms_cols = [
    'Visit1_heartburn', 'Visit1_nausea', 'Visit1_vomiting', 'Visit1_diarrhea', 'Visit1_abdominal_pain', 'Visit1_dyspepsia',
    'Visit2_heartburn', 'Visit2_nausea', 'Visit2_vomiting', 'Visit2_diarrhea', 'Visit2_abdominal_pain', 'Visit2_dyspepsia',
    'Visit3_heartburn', 'Visit3_nausea', 'Visit3_vomiting', 'Visit3_diarrhea', 'Visit3_abdominal_pain', 'Visit3_dyspepsia',
    'Visit4_heartburn', 'Visit4_nausea', 'Visit4_vomiting', 'Visit4_diarrhea', 'Visit4_abdominal_pain', 'Visit4_dyspepsia',
    'Visit5_heartburn', 'Visit5_nausea', 'Visit5_vomiting', 'Visit5_diarrhea', 'Visit5_abdominal_pain', 'Visit5_dyspepsia'
]

symptoms_data = df[symptoms_cols]

# 이진 변수로 변환 (증상 있음: 1, 없음: 0)
symptoms_data_binary = symptoms_data.applymap(lambda x: 1 if x > 0 else 0)

# 증상 별 합산
symptoms_sums = symptoms_data_binary.groupby(lambda x: x.split('_')[0], axis=1).sum()

# 개선 여부 비교 함수
def compare_symptoms(df, visit1, visit2):
    before_symptoms = df[f'Visit{visit1}'] > 0
    after_symptoms = df[f'Visit{visit2}'] > 0
    return pd.crosstab(before_symptoms, after_symptoms)

# 방문 간 증상 개선 여부 비교 및 맥니마 검정 수행
results = {}
for i in range(1, 5):
    table = compare_symptoms(symptoms_sums, i, i+1)
    result = mcnemar(table, exact=True)
    results[f'Visit{i} vs Visit{i+1}'] = (result.statistic, result.pvalue)

# 결과 출력
for comparison, (stat, p) in results.items():
    print(f"{comparison} - Statistic: {stat}, p-value: {p}")

# p-value 시각화
comparisons = list(results.keys())
p_values = [results[comp][1] for comp in comparisons]

plt.figure(figsize=(10, 6))
plt.plot(comparisons, p_values, marker='o', linestyle='-', color='skyblue')
plt.axhline(y=0.05, color='r', linestyle='--', label='Significance Level (0.05)')
plt.title('McNemar Test p-values for Overall GI Symptoms')
plt.xlabel('Visit Comparison')
plt.ylabel('p-value')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
df.loc[:,symptoms_cols]

In [ ]:
excl_cols=['No','Name','이니셜','BD','ICF signed Date','site','alcohol','/week','how','MI_1','angina_1'
           ,'Lab_Date','중도탈락 여부','마지막 f/u','중도탈락 사유'
           ]
when_cols=[col for col in df.columns if col.startswith('When')]
smoking_cols=[col for col in df.columns if col.startswith('smoking')]
aspirin_cols=[col for col in df.columns if col.endswith('처방')]
rest_cols=\
    [col for col in df.columns if col.endswith('hematochezia')]\
    +[col for col in df.columns if col.endswith('melena')]\
    +[col for col in df.columns if col.endswith('hematemesis')]\
    +[col for col in df.columns if col.startswith('hospitalization')]\
    +[col for col in df.columns if col.startswith('bleeding')]\
    +[col for col in df.columns if col.endswith('_date')]\
    +[col for col in df.columns if col.endswith('heartburn')]\
    +[col for col in df.columns if col.endswith('vomiting')]\
    +[col for col in df.columns if col.startswith('Lab_Date')]
date_cols=[col for col in df.columns if col.startswith('hospitalization_date')]

In [ ]:
cox_df=df.drop(columns=excl_cols+when_cols+smoking_cols+aspirin_cols+rest_cols+date_cols)

In [ ]:
cox_df.isna().sum().sort_values()

In [ ]:
cox_df.dropna()

In [ ]:
cox_df=cox_df.apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [ ]:
id_vars=['ID','Age','gender','height','weight','AF','angina','MI','CI','HTN','DM','HF','DL','TIA','stroke/TIA','Ischemic stroke','Hemorrhagic stroke','systemic embolism','Peripheral artery disease','GI Bleeding history','PCI','previous bleeding']

In [ ]:
valcols=[col for col in cox_df.columns if col.startswith('Visit') and not (col.endswith('DATE')|col.endswith('ING'))]
meldf=[]
for subdf in list(cox_df.melt(id_vars=id_vars).groupby('ID')):
    subdf=pd.concat([subdf[1].drop(columns='variable'),subdf[1].variable.apply(lambda x:pd.Series(x.split('_',1),index=['Visit','Outcome']))],axis=1)
    subdf.Visit=subdf.Visit.str[-1].astype(np.int8)
    meldf.append(pd.merge(subdf.drop(columns=['Outcome','value']).groupby('Visit').max(),subdf.pivot_table(index='Visit',columns='Outcome',values='value'),'right','Visit').reset_index())
meldf=pd.concat(meldf).reset_index(drop=True)
valcols=list(set(map(lambda x:x.split('_',1)[-1],valcols)))
cox_df_=meldf

In [ ]:
pd.Series(symptoms_cols).str.split('_').str[1].unique()

In [ ]:
cox_df_.loc[:,['nausea','diarrhea','dyspepsia']]

In [ ]:
cox_df_.loc[(cox_df_.loc[:,'dyspepsia']==1)].index,\
cox_df_.loc[(cox_df_.loc[:,'diarrhea']==1)].index,\
cox_df_.loc[(cox_df_.loc[:,'nausea']==1)].index

In [ ]:
cox_df_=cox_df_.rename(columns={'diarrhea':'event'})

In [ ]:
cox_df_.loc[cox_df_.event==1,:]

In [ ]:
# p-value 표기법
# p>0.05 : ns / p<=0.05 : * / p<=0.01 : ** / p<=0.001 : *** / p<=0.0001 : ****
def pround(pval, precision=2, asterisk=False):
    if pval>=0.05: rslt=('p={{:.{}f}}'.format(precision).format(pval).replace('0.','.'), '')
    elif pval>=0.01: rslt=('p<.05', '*')
    elif pval>=0.001: rslt=('p<.01', '**')
    elif pval>=0.0001: rslt=('p<.001', '***')
    else: rslt=('p<.0001', '****')
    if asterisk: return rslt
    else: return rslt[0]

In [ ]:
# 시변 콕스 회귀 모델
# baseline_val=cox_df_.loc[:,['PAT_NO','start',event_name]].groupby('PAT_NO').apply(lambda x:x.loc[x.start==0,event_name].item())
init=True
# event_col=pd.Series(cox_df_.VR<=80,name='event')
event_col=pd.Series(cox_df_.event)

ctv=lf.CoxTimeVaryingFitter(penalizer=.1)
if init: target_cols, passed_cols=[],[]
for col in [col for col in cox_df_.columns if col not in ['Visit','ID','end','event',]]:
    col=[col]
    try:
        if not init: col=[]
        ctv.fit(
            pd.concat([cox_df_.drop(columns=['event']), pd.Series(cox_df_.Visit+1,name='end'), event_col],axis=1).loc[:,
            # pd.concat([cox_df_, pd.Series(cox_df_.Visit+1,name='end')],axis=1).loc[:,
            # pd.concat([cox_df_, pd.Series(cox_df_.Visit+1,name='end'), event_col],axis=1)\
            #     .groupby('PAT_NO').apply(drop_after_event).drop(columns='PAT_NO').reset_index().loc[:,
                    ['Visit','ID','end','event']+target_cols+col
                ].dropna(),
            id_col='ID',
            event_col='event',
            show_progress=True,
            start_col='Visit',
            stop_col='end',
        )
        if init: target_cols.append(*col)
        else: break
    except ValueError:
        passed_cols.append(*col)
plt.figure(figsize=(6,12))
# plt.xlim(-4,4)
ctv.plot()
plt.tight_layout()
plt.show()
# ctv.print_summary()
rslt=ctv.summary.sort_values('p',ascending=True)
rslt.p=rslt.p.map(pround)
rslt